In [6]:
import itertools as it
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm
from sklearn.gaussian_process.kernels import Matern, RBF

# Function 5

We begin this guide by downloading the data:

In [7]:
X_org = np.load('initial_inputs.npy')
Y_org = np.load('initial_outputs.npy')


In [8]:
prev_runs = np.array([[0.750000,0.999999,0.999999,0.999999],
                      [0.999999,0.999999,0.999999,0.999999],
                      [0.999999,0.759999,0.899999,0.999999],
                      [0.799999,0.759999,0.899999,0.799999],
                      [0.999999,0.999999,0.999999,0.999999],
                      ])
X=np.append(X_org, prev_runs, axis=0)
y=np.append(Y_org, [5652.14556780706,
                    8662.40500124829,
                    4444.94497355769,
                    1285.04936469052,
                    8662.405001248297
                    ], axis=0)

### Random Search

The simplest solution would be a simple random search, that is, we can randomly choose our next query point:

In [9]:
next_query = np.random.uniform(size = 4)
print(next_query)

[0.1855754  0.82098109 0.26717067 0.22081939]


While this solution is easy to implement, we know it will be very slow. However, it could serve as a placeholder for gathering more information while you research which method you want to use for each function.

### Upper Confidence Bound

A second alternative would be to use Bayesian Optimization and consider an Upper Confidence Bound acquisition function: 

In [10]:
# Define Gaussian Process model with RBF kernel
kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gpr = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
#
# gpr = GaussianProcessRegressor(kernel=None)
gpr.fit(X, y)

GaussianProcessRegressor(alpha=1e-06, kernel=1**2 * RBF(length_scale=1),
                         normalize_y=True)

In [11]:
def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.01):
    mu, sigma = gpr.predict(X, return_std=True)
    mu_sample = gpr.predict(X_sample)
    
    mu_sample_opt = np.max(mu_sample)
    
    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0  # Ensure correct dimensions
    
    return ei

In [12]:
def upper_confidence_bound(X, gpr, kappa=2.5):
    mu, sigma = gpr.predict(X, return_std=True)
    ucb = mu + kappa * sigma
    return ucb

In [13]:
# to optimize the acquisition function, we will simply use gridsearch over a space of 10.000 gridpoints
x1 = np.linspace(0, 1, 5)

In [14]:
x1

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [15]:
import itertools as it

In [16]:
list(it.product(['1','2','3','4',], ['a', 'b','c','d']))

[('1', 'a'),
 ('1', 'b'),
 ('1', 'c'),
 ('1', 'd'),
 ('2', 'a'),
 ('2', 'b'),
 ('2', 'c'),
 ('2', 'd'),
 ('3', 'a'),
 ('3', 'b'),
 ('3', 'c'),
 ('3', 'd'),
 ('4', 'a'),
 ('4', 'b'),
 ('4', 'c'),
 ('4', 'd')]

In [17]:
dim = 4
X_grid = np.fromiter(it.chain(*it.product(x1, repeat=dim)), dtype=float).reshape(-1,dim)

In [18]:
X_grid.shape

(625, 4)

In [19]:
# Evaluate the acquisition function on the grid
ei = expected_improvement(X_grid, X, y, gpr)
# Find the next query point
next_query_ei = X_grid[np.argmax(ei)]
print('Next query point:', next_query_ei)

Next query point: [1.   1.   0.75 1.  ]


In [21]:
# Evaluate the acquisition function on the grid
ucb = upper_confidence_bound(X_grid, gpr, kappa=2.5)

# Ensure correct shape for plotting
#ucb = ucb.reshape(no_of_samples, no_of_samples, no_of_samples)

# Find the next query point
next_query = X_grid[np.argmax(ucb)]
print('Next query point:', next_query)

Next query point: [1.   1.   0.75 1.  ]


In [24]:
print(f"{str(next_query[0]).ljust(8,'0')}-{str(next_query[1]).ljust(8,'0')}-{str(next_query[2]).ljust(8,'0')}-{str(next_query[3]).ljust(8,'0')}")

1.000000-1.000000-0.750000-1.000000


## Visualizing our data and thinking of the problem

It is important when tackling problems to really think about the best strategy and to do some exploratory data analysis. Let's consider what we know about the problem:

1. From the hints, we expect two modes in the unknown function.

2. From the hints, we know that most of our queries should be zero!

3. The problem is two-dimensional.

4. The problem will have small length-scale (that is, we expect the modes to be very small)

From (3.) we can take advantage, and plot the initial data:

In [ ]:
fig, ax = plt.subplots()
fig.set_figheight(5)
fig.set_figwidth(8)
plt.scatter(X[:, 0], X[:, 1], c = Y)
plt.colorbar();